# Run the scVI method on the Paul data set

We can only get 8GB of RAM to work with a GPU.  Thus, to run this, we must (in general): 

1) Load the data for a fold and fit the scVI model

2) Get the needed information for feature selection from the scVI model.  This is, in general, a very large, dense matrix.  I call it the `stuff` in the following.

3) Load the `stuff` when you are not running on GPUs so that you have more memory to play with.  Then use scvi and the `stuff` to find the markers and the "p values"

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

In [3]:
import scanpy.api as sc

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, color_map='viridis')  # low dpi (dots per inch) yields small inline figures
sc.logging.print_versions()

scanpy==1.3.2+25.g8ac9c03 anndata==0.6.11 numpy==1.14.6 scipy==1.1.0 pandas==0.23.4 scikit-learn==0.20.0 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


In [4]:
import sys
sys.path.append('/home/ahsvargo/xvalid')
sys.path.append('/home/ahsvargo/.local/bin')

In [5]:
from picturedrocks import Rocks
from picturedrocks.performance import FoldTester, PerformanceReport, NearestCentroidClassifier

## Load all of the data

Don't need to do this in general.  We have made separate data files for each fold further below

In [44]:
adata = sc.datasets.paul15()

... storing 'paul15_clusters' as categorical


In [45]:
sc.logging.print_memory_usage()

Memory usage: current 2.42 GB, difference +2.42 GB


In [46]:
adata

AnnData object with n_obs × n_vars = 2730 × 3451 
    obs: 'paul15_clusters'
    uns: 'iroot'

In [8]:
adata.write("paul15-scanpy.h5ad")

Sparsity:

In [274]:
((adata.X != 0.0)*1.0).sum()/(adata.X.shape[0]*adata.X.shape[1])

0.253284125321216

In [47]:
lookup = list(adata.obs['paul15_clusters'].cat.categories)
yVec = np.array([lookup.index( adata.obs['paul15_clusters'][i] ) for i in range(adata.obs['paul15_clusters'].shape[0]) ])

In [48]:
# paul
data = Rocks(adata.X, yVec)

## Make some folds

Make some new folds for scVI testing.  These functions copied from the PicturedRocks class

In [20]:
def kfoldindices(n, k, random=False):
    basearray = np.arange(n)
    if random:
        np.random.shuffle(basearray)
    lengthfloor = n//k
    extra = n % k
    cur = 0
    while cur < n:
        thislength = lengthfloor + 1 if extra > 0 else lengthfloor
        yield basearray[cur:cur + thislength]
        cur += thislength
        extra -= 1 # this should be extra = max(extra - 1, 0),
        #            but it doesn't matter
        
def makefolds(N, k=5, random=False):
    folds = list(kfoldindices(N, k, random))
    return folds

In [22]:
myFolds = makefolds(data.N, random=True)

In [63]:
ft = FoldTester(data)
#ft.loadfolds("paul/paul15-scviFolds.npz")

In [64]:
ft.folds = myFolds

In [65]:
ft.validatefolds()

True

Save the folds as well

In [28]:
type(myFolds)

list

In [29]:
np.savez("paul15-scviFolds", folds=np.array(myFolds))

## Make the data files

Need a separate file for each fold in the data - we need to run each fold separately since otherwise we use too much memory.  Remember that we train on the data this is NOT in the fold.

In [51]:
myFolds = np.load("paul15-scviFolds.npz")["folds"]

In [18]:
for i,fold in enumerate(myFolds):
    mask = np.zeros(data.N, dtype=bool)
    mask[fold] = True
    
    fname = "paul15-fold" + str(i) + ".h5ad"
    print("Writing fold {} to file {}".format(i,fname), flush=True)
    currAdata = adata[~mask]
    print(currAdata.X.shape, flush=True)
    
    currAdata.write(fname)

Writing fold 0 to file paul15-fold0.h5ad
(2184, 3451)
Writing fold 1 to file paul15-fold1.h5ad
(2184, 3451)
Writing fold 2 to file paul15-fold2.h5ad
(2184, 3451)
Writing fold 3 to file paul15-fold3.h5ad
(2184, 3451)
Writing fold 4 to file paul15-fold4.h5ad
(2184, 3451)


## Start the scVI stuff and make the model

The Paul dataset is small enough that we can collect all of the `stuff` for the folds in 8GB without restarting the kernel (as long as we delete the `stuff` at each step).  

In [11]:
import scvi
from scvi.dataset import AnnDataset
from scvi.models import *
from scvi.inference import UnsupervisedTrainer

In [32]:
myFolds = np.load("paul15-scviFolds.npz")["folds"]

In [109]:
currFold=4

In [110]:
data = AnnDataset("paul15-fold{}.h5ad".format(currFold), save_path="paul/")

File paul/paul15-fold4.h5ad already downloaded
Preprocessing dataset
Finished preprocessing dataset


In [111]:
data.gene_names

array(['0610007L01Rik', '0610009O20Rik', '0610010K14Rik', ...,
       'mKIAA1994', 'rp9', 'slc43a2'], dtype='<U18')

### Side note: the scVI object does not re-order the genes (as far as I can tell)

Thus, we can use the indices of the genes that scVI chooses as the markers in our marker evaluations.

In [112]:
np.all(np.array(adata.var.index) == data.gene_names)

True

### Continuting: finish setting up the scVI object

The number of samples - we need to input the labels as well.

In [113]:
data.labels.shape

(2184, 1)

In [114]:
np.unique(data.labels)

array([0.])

In [115]:
mask = np.zeros(adata.X.shape[0], dtype=bool)
mask[myFolds[currFold]] = True
data.labels = yVec[~mask].reshape((2184,1))

In [116]:
np.all(adata.X[~mask] == data.X)

True

Ideally, each fold will contain samples from each cluster.  Sometimes, this is not possible.  But we should check anyways.  Need to tell the scVI object the number of labels.

In [118]:
np.unique(yVec[~mask]).shape

(19,)

In [120]:
data.n_labels = 19

In [121]:
n_epochs=400
lr=1e-3
use_batches=False
use_cuda=True

In [122]:
import torch
torch.cuda.is_available()

True

In [123]:
torch.cuda.device_count()

1

In [124]:
vae = VAE(data.nb_genes, n_batch=data.n_batches * use_batches)
trainer = UnsupervisedTrainer(vae,
                              data,
                              train_size=0.75,
                              use_cuda=use_cuda,
                              frequency=5)
trainer.train(n_epochs=n_epochs, lr=lr)

training: 100%|██████████| 400/400 [01:15<00:00,  4.92it/s]


## find "p values" for their fold

In [125]:
stuff=None

In [126]:
%%time
stuff = trainer.train_set.differential_expression_stats()

CPU times: user 9.49 s, sys: 1.12 s, total: 10.6 s
Wall time: 10.6 s


In [127]:
trainer.train_set.data_loader_kwargs['batch_size']

128

In [128]:
stuff[0].shape

(163800, 3451)

In [129]:
type(stuff[1])

numpy.ndarray

Save the `stuff` so that we can load it later for further analysis if needed.

In [130]:
np.savez("paul-scvi-fold{}-stuff".format(currFold), stuff0=stuff[0], stuff1=stuff[1])

### Try to get an idea of how large the `stuff` object will be.

The bold headings below correspond to the data that was run with the correct fold structure (training on the points that aren't in the fold).

I can't see any definite patterns.  Larger clusters get more samples in `stuff`

**Fold 0**:

In [29]:
np.unique(stuff[1], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18]),
 array([ 2400, 18800, 14100,  8200,  9700,  9700, 10300,  3700,  3400,
         9300,  1400,  4500, 18700, 24600, 11200, 10500,  1200,   500,
         1600]))

In [30]:
np.unique(yVec[~mask], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18]),
 array([ 34, 261, 198, 102, 135, 141, 138,  55,  45, 125,  21,  57, 241,
        301, 138, 140,  18,   7,  27]))

Fold 1:

In [160]:
np.unique(stuff[1], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18]),
 array([ 600, 5400, 3400, 1700, 1900, 2800, 2100, 1400, 1100, 2500,  300,
        1500, 4500, 5300, 2900, 2600,  300,  100,  500]))

In [162]:
np.unique(yVec[myFolds[1]], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18]),
 array([ 7, 78, 52, 25, 32, 34, 23, 16, 16, 30,  3, 16, 60, 71, 36, 32,  4,
         2,  9]))

Fold 2:

In [185]:
np.unique(stuff[1], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        18]),
 array([ 500, 4500, 2900, 1500, 2900, 2600, 2800,  600,  800, 2700,  700,
        1400, 5000, 5500, 3000, 2300,  400,  800]))

In [186]:
np.unique(yVec[myFolds[2]], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18]),
 array([ 7, 62, 49, 22, 36, 36, 32,  8, 10, 34,  7, 16, 63, 76, 38, 35,  5,
         2,  8]))

**Fold 3**:

In [107]:
np.unique(stuff[1], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18]),
 array([ 2400, 19400, 15900,  7100, 11500, 10700,  8300,  3900,  3900,
        10100,  1700,  3700, 17400, 22400, 11200, 10100,   900,   700,
         2500]))

In [108]:
np.unique(yVec[~mask], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18]),
 array([ 34, 267, 202,  96, 147, 141, 118,  54,  52, 123,  22,  53, 242,
        295, 157, 130,  15,   8,  28]))

**Fold 4**:

In [131]:
np.unique(stuff[1], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18]),
 array([ 2500, 20600, 14700,  7600, 10700, 10200,  9300,  3900,  3700,
         9400,  2100,  4600, 17500, 22700, 10800,  9300,  1600,   500,
         2100]))

In [132]:
np.unique(yVec[~mask], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18]),
 array([ 32, 270, 193,  97, 146, 134, 133,  51,  55, 122,  27,  60, 240,
        297, 151, 125,  20,   7,  24]))

In fold 2: We don't even sample for cluster 17... not sure how this works at all.  Certainly can't find markers for cluster 17 in this case.  I don't think this matters too much for our analysis but still... not the greatest situation

The size of the matrix seems to be constant depending on the number of genes, but spread around the different clusters in a different way each time.  The way that it is spread out depends on the number of sample points from the cluster

In [133]:
np.unique(yVec, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18]),
 array([ 43, 329, 246, 124, 180, 173, 167,  68,  63, 153,  30,  69, 300,
        373, 186, 164,  22,   9,  31]))

## Process the `stuff` to find the p-values and markers

You need about 8 GB to run this next part... Probably restart the kernel and run off of gpus if you want to do the following:

In [8]:
dayta = np.load("paul15-scvi-markerData.npz")
stuff = (dayta['mat'], dayta['vec'])

In [6]:
M_sampling = 100

M_permutation = 100000
permutation = False

In [35]:
currFold = 4

In [36]:
fname = "paul-scvi-fold{}-stuff.npz".format(currFold)
file = np.load(fname)
stuff = (file["stuff0"], file["stuff1"])
print("Read data from {}".format(fname))

Read data from paul-scvi-fold4-stuff.npz


In [37]:
stuff[0].shape

(163800, 3451)

Markers for one cluster:

In [222]:
%%time
pvals0 = scvi.inference.posterior.get_bayes_factors(stuff[0], stuff[1], 0, M_permutation=M_permutation,
                                                   permutation=permutation)

CPU times: user 2.25 s, sys: 1.78 s, total: 4.03 s
Wall time: 4.04 s


Markers for all clusters:

In [38]:
%%time
allMarks = []
allPvals = []

for clust in np.unique(stuff[1]):
    print(clust, flush=True)
    
    pvals = scvi.inference.posterior.get_bayes_factors(stuff[0], stuff[1], clust, M_permutation=M_permutation,
                                                   permutation=permutation)
    markers = np.flipud(np.argsort(np.abs(pvals)))
    allMarks.append(markers)
    
    allPvals.append(pvals)
    
allMarks=np.array(allMarks)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
CPU times: user 1min 34s, sys: 1min 13s, total: 2min 48s
Wall time: 2min 49s


In [39]:
allMarks

array([[1421, 1422,  472, ..., 1352, 2530, 3295],
       [2186, 1121, 1191, ..., 1798,  503, 1548],
       [1978, 2290,  302, ..., 1714, 3273, 2514],
       ...,
       [2391, 1842,  402, ..., 2270, 2674, 1184],
       [ 262, 1745, 1756, ...,   53, 2972, 1752],
       [ 627,  609, 2758, ...,  928, 2234, 3302]])

In [40]:
np.savez("paul15-scvi-marks-fold{}".format(currFold), allMarks)
np.savez("paul15-scvi-pvals-fold{}".format(currFold), allPvals)

In [75]:
pvals0[markers[102]]

-3.2278776806652867

In [58]:
markers = np.flipud(np.argsort(np.abs(pvals0)))

Looking at the cutoff method for selecting markers, as suggested in the scVI paper

In [67]:
mini = 0
for i in range(markers.shape[0]):
    if np.abs(pvals0[markers[i]]) >= 3.0:
        if i > mini:
            mini = i
            
print(mini)
        

141


In [99]:
adata

AnnData object with n_obs × n_vars = 2730 × 3451 
    obs: 'paul15_clusters'
    uns: 'iroot'

## Just look at these markers a little bit with picturedRocks

We have not done any cross-validation, so we can't really make any conclusions based on this.  But it is nice to see it anyways.

In [101]:
from picturedrocks import Rocks
from picturedrocks.performance import FoldTester, PerformanceReport, NearestCentroidClassifier

In [102]:
test = Rocks(adata.X, yVec)

In [103]:
test.normalize(totalexpr=10000, log=True)

In [264]:
ft = FoldTester(test)
ft.folds=myFolds

In [201]:
marks_list = [2,4,6,8,10,12,14,16,18,20,25,30,35,40,45,50,55,60]
xlist = []
ylist = []

for marks_per_clust in marks_list:
    
    print("markers per cluster: {}".format(marks_per_clust))
    myMarks = [list(table[:marks_per_clust]) for table in allMarks]
    myMarks = list(set().union(*myMarks))
    
    xlist.append(len(myMarks))
    
    cla = NearestCentroidClassifier()
    cla.train(Rocks(test.X[:, myMarks], test.y))
    yTest = cla.test(test.X[:, myMarks], False)
    
    ylist.append(np.sum( (yVec.flatten() != yTest)*1.0))

markers per cluster: 2
markers per cluster: 4
markers per cluster: 6
markers per cluster: 8
markers per cluster: 10
markers per cluster: 12
markers per cluster: 14
markers per cluster: 16
markers per cluster: 18
markers per cluster: 20
markers per cluster: 25
markers per cluster: 30
markers per cluster: 35
markers per cluster: 40
markers per cluster: 45
markers per cluster: 50
markers per cluster: 55
markers per cluster: 60


Compare the the values in `graphs.ipynb` to see how these compare.  This looks really bad.  

In [202]:
np.array(ylist, dtype=int)

array([1597, 1228, 1062, 1048,  989,  959,  928,  890,  866,  851,  808,
        752,  730,  719,  710,  698,  678,  655])

In [203]:
np.array(xlist)

array([ 36,  72, 106, 141, 177, 211, 246, 279, 310, 343, 425, 504, 583,
       658, 731, 810, 882, 942])

In [140]:
test.X.shape

(2730, 3451)

So scVI seems to perform terribly on this data set.  Not sure what's going on here...

Investigating more: it actually does quite well when you include more genes.  Not sure how well, since there is no cross-validation.  Take care of that soon.  But.  We (essentially) monotonically converge to the base rate as we increase the number of markers that we are looking at.

It does seem to do worse when we are looking at small numbers of markers, however.  Interesting stuff.

Explore with the markers from one cluster - are we making mistakes?  We don't get many more cells correct when we include the markers for all clusters...

In [69]:
cla = NearestCentroidClassifier()
numMarks = 141
currMarks = markers[:numMarks]

cla.train(Rocks(test.X[:, currMarks], test.y))
yTest = cla.test(test.X[:, currMarks], False)

In [70]:
np.sum( (yVec.flatten() != yTest)*1.0)/2730.

0.41794871794871796

In [71]:
np.sum( (yVec.flatten() != yTest)*1.0)

1141.0

## The full cross-validation analysis

Finding the number of errors using the Rocks object.  You don't need to run this - all of this now taken care of in `pvalue-classification-errors.ipynb` 

In [41]:
path="./"
pvalMarkerList = []
for fold in range(5):
    dayta = np.load(path + "paul15-scvi-marks-fold" + str(fold) + ".npz")
    pvalMarkerList.append(dayta['arr_0'])

In [42]:
len(pvalMarkerList)

5

`data` is a PicturedRocks object that is created at the start of this notebook

In [62]:
data.normalize(totalexpr=10000, log=True)

In [250]:
 [list(set().union(*table[:,:2])) for table in pvalMarkerList]

[[647,
  3211,
  2707,
  1953,
  162,
  1450,
  2093,
  2223,
  561,
  819,
  1976,
  2937,
  1597,
  1981,
  1727,
  2496,
  1346,
  1870,
  724,
  1876,
  1751,
  2264,
  2647,
  1367,
  986,
  2912,
  2787,
  100,
  1386,
  3179,
  3052,
  2667,
  619,
  1774,
  2290,
  2418,
  631,
  3449],
 [2307,
  2824,
  137,
  266,
  136,
  142,
  1935,
  527,
  2590,
  1827,
  2473,
  1962,
  2729,
  1193,
  1709,
  302,
  2350,
  1588,
  1212,
  1344,
  2245,
  2250,
  2763,
  2642,
  220,
  1119,
  2912,
  225,
  2400,
  2273,
  1764,
  3429,
  3052,
  365,
  1524,
  1531,
  2301,
  1406],
 [2435,
  1284,
  1421,
  1422,
  2959,
  911,
  1429,
  2331,
  162,
  2851,
  803,
  552,
  1705,
  173,
  949,
  1975,
  2361,
  1978,
  316,
  3261,
  190,
  2113,
  2116,
  2893,
  80,
  1745,
  856,
  475,
  221,
  1503,
  483,
  1636,
  613,
  1258,
  2165,
  2295],
 [262,
  1550,
  3219,
  790,
  793,
  413,
  1949,
  1438,
  290,
  36,
  2729,
  171,
  1707,
  440,
  827,
  188,
  3390,
  713,
  

In [254]:
[table[:,:2] for table in pvalMarkerList][0]

array([[1386, 2707],
       [1597,  631],
       [2290, 2912],
       [1870,  561],
       [3449, 1953],
       [1751, 3211],
       [ 724, 3052],
       [2787, 3179],
       [1976, 1346],
       [2264, 2418],
       [2667, 1727],
       [2496, 1450],
       [ 162,  619],
       [2093, 1876],
       [2223, 2647],
       [1774, 1367],
       [ 647, 1981],
       [2937,  986],
       [ 819,  100]])

In [259]:
[list(set().union(*table[:,:2])) for table in pvalMarkerList]

[[647,
  3211,
  2707,
  1953,
  162,
  1450,
  2093,
  2223,
  561,
  819,
  1976,
  2937,
  1597,
  1981,
  1727,
  2496,
  1346,
  1870,
  724,
  1876,
  1751,
  2264,
  2647,
  1367,
  986,
  2912,
  2787,
  100,
  1386,
  3179,
  3052,
  2667,
  619,
  1774,
  2290,
  2418,
  631,
  3449],
 [2307,
  2824,
  137,
  266,
  136,
  142,
  1935,
  527,
  2590,
  1827,
  2473,
  1962,
  2729,
  1193,
  1709,
  302,
  2350,
  1588,
  1212,
  1344,
  2245,
  2250,
  2763,
  2642,
  220,
  1119,
  2912,
  225,
  2400,
  2273,
  1764,
  3429,
  3052,
  365,
  1524,
  1531,
  2301,
  1406],
 [2435,
  1284,
  1421,
  1422,
  2959,
  911,
  1429,
  2331,
  162,
  2851,
  803,
  552,
  1705,
  173,
  949,
  1975,
  2361,
  1978,
  316,
  3261,
  190,
  2113,
  2116,
  2893,
  80,
  1745,
  856,
  475,
  221,
  1503,
  483,
  1636,
  613,
  1258,
  2165,
  2295],
 [262,
  1550,
  3219,
  790,
  793,
  413,
  1949,
  1438,
  290,
  36,
  2729,
  171,
  1707,
  440,
  827,
  188,
  3390,
  713,
  

In [253]:
pvalMarkerList[0].shape

(19, 3451)

In [74]:
%%time
marks_list = [2,4,6,8,10,12,14,16,18,20,25,30,35,40,45,50,55,60]
xlist = []
ylist = []

for marks_per_clust in marks_list:
    
    print("markers per cluster: {}".format(marks_per_clust))
    myMarks = [list(set().union(*table[:,:marks_per_clust])) for table in pvalMarkerList]
    ft.markers = myMarks
    print("total markers: {}".format([len(a) for a in ft.markers]))
    
    xlist.append(np.array([len(a) for a in ft.markers]).mean())
    
    ft.classify(NearestCentroidClassifier)
    ylist.append(np.sum( (yVec.flatten() != ft.yhat)*1.0))

markers per cluster: 2
total markers: [38, 37, 38, 38, 38]
markers per cluster: 4
total markers: [76, 72, 75, 75, 76]
markers per cluster: 6
total markers: [114, 108, 112, 112, 110]
markers per cluster: 8
total markers: [151, 145, 149, 148, 144]
markers per cluster: 10
total markers: [186, 181, 183, 184, 180]
markers per cluster: 12
total markers: [221, 216, 218, 218, 217]
markers per cluster: 14
total markers: [257, 250, 252, 254, 251]
markers per cluster: 16
total markers: [290, 286, 288, 290, 284]
markers per cluster: 18
total markers: [324, 322, 321, 321, 313]
markers per cluster: 20
total markers: [356, 354, 353, 356, 345]
markers per cluster: 25
total markers: [432, 433, 433, 434, 425]
markers per cluster: 30
total markers: [507, 506, 509, 516, 497]
markers per cluster: 35
total markers: [582, 583, 585, 597, 566]
markers per cluster: 40
total markers: [650, 653, 654, 676, 645]
markers per cluster: 45
total markers: [723, 723, 721, 748, 716]
markers per cluster: 50
total markers: 

In [72]:
np.array(ylist)

array([1564., 1308., 1142., 1071., 1029.,  998.,  965.,  982.,  961.,
        948.,  920.,  883.,  875.,  882.,  881.,  870.,  847.,  848.])

In [73]:
np.array(xlist)

array([ 37.8,  74.8, 111.2, 147.4, 182.8, 218. , 252.8, 287.6, 320.2,
       352.8, 431.4, 507. , 582.6, 655.6, 726.2, 795.4, 860.6, 925.2])